In [1]:
import numpy as np

In [3]:
def create_input_data_file(samples_count, modulo, file_name="test_input.txt", rand_state=1):
    """ Create text file with samples of input data for testbench.
    
        Every string has format: <data_1> <data_2> <modulo>, where <.> - integer >= 0
        Arguments:
        - samples_count - number of strings in file, integer > 0;
        - modulo - modulo, array [samples_count, 1] of modulo for every input or integer - one module for all inputs;
        - file_name - string with name of created file
        - rand_state - state of random generator, for samples repeatability
        
        return array [samples_count, 3] of integers where each row has an appropriate string content from file.
    """
    assert type(modulo) in [int, np.ndarray], "Error! Modulo must be integer or array."
    assert isinstance(modulo, int) or (modulo.size == len(modulo) == samples_count), "Error! If modulo is array " +\
        "then it must be vector with 'samples_count' length."
    
    # generate array with samples
    rgen = np.random.RandomState(seed=rand_state)
    if isinstance(modulo, np.ndarray):
        modulo = modulo.reshape(-1, 1)
        max_modulo = np.max(modulo)
        data = rgen.randint(0, max_modulo - 1, (samples_count, 2))
        mod_data = np.hstack((np.mod(data, modulo), modulo)).astype(int)
    elif isinstance(modulo, int):
        data = rgen.randint(0, modulo - 1, (samples_count, 2))
        modulo_vec = modulo * np.ones((samples_count, 1))
        mod_data = np.hstack((data, modulo_vec)).astype(int)
        
    # write sample to file        
    with open(file_name, 'w') as f:
        strings = []
        for ii in range(samples_count):
            strings.append(f"{mod_data[ii, 0]:d} {mod_data[ii, 1]:d} {mod_data[ii, 2]:d}\n")
        f.writelines(strings)
    return mod_data

def create_output_data_file(input_data_file="test_input.txt", output_data_file="test_output.txt"):
    """ Create text file with samples of output data for testbench.
    
        Every string has format: <out_data>, where out_data - integer >= 0
        Arguments:
        - input_data_file - name (destination) of file with input data, string. Before execute function 
            you must create this file (using create_input_data_file(), for example) first.
        - output_data_file - name (destination of created file with output data). File contains the sum of samples
        <data_1> and <data_2> modulo <modulo> from input_data_file in each string, see create_input_data_file() help.
    
        return array [samples_count, 1] with sums written to file output_data_file.
    """
    try:
        with open(input_data_file, 'r') as f_rd:
            with open(output_data_file, 'w') as f_wr:
                out_data = None
                for input_string in f_rd.readlines():
                    data_1, data_2, modulo = (int(el) for el in input_string.split(" "))
                    output_string = np.array(np.mod(data_1 + data_2,  modulo))
                    if out_data is not None:
                        out_data = np.vstack((out_data, output_string))
                    else:
                        out_data = np.copy(output_string)
                f_wr.writelines((f"{int(el)}\n" for el in out_data))
        return out_data
    except IOError:
        print(f"Error! File {input_data_file} is not accessible.")
            

In [3]:
show_files = False
check_files = True
samples_count = 2000
modulos = [50, 100, 200, 256]
modulo_vec = None
for cur_modulo in modulos:
    new_modulos = cur_modulo * np.ones(samples_count // len(modulos))
    if modulo_vec is not None:
        modulo_vec = np.hstack((modulo_vec, new_modulos))
    else:
        modulo_vec = np.copy(new_modulos)
        
in_data = create_input_data_file(samples_count=samples_count, modulo=modulo_vec, file_name="test_input.txt", rand_state=1)

if show_files:
    with open("test_input.txt", 'r') as f:
        for data_str in f.readlines():
            print(data_str)

In [4]:
out_data = create_output_data_file(input_data_file="test_input.txt", output_data_file="test_output.txt")

if show_files:
    with open("test_output.txt", 'r') as f:
        for data_str in f.readlines():
            print(data_str)

In [5]:
if check_files:
    assert np.all(np.mod(in_data[:, 0] + in_data[:, 1], in_data[:, 2]).reshape(out_data.shape) == out_data), \
        "Error! Data from output_file don't match data from input_file."